In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta, timezone
import time
import os 
from zoneinfo import ZoneInfo

In [29]:
ROUTE_ID = "Orange"
OUTPUT_SCHEDULE = "orange_line_schedule_live.csv"
OUTPUT_REALTIME = "orange_line_realtime_live.csv"

COLLECTION_INTERVAL_MINUTES = 1
TOTAL_DURATION_HOURS = 9      
ITERATIONS = int((TOTAL_DURATION_HOURS * 60) / COLLECTION_INTERVAL_MINUTES)
eastern = ZoneInfo("America/New_York")

In [3]:
def collect_mbta_data():
    eastern = ZoneInfo("America/New_York")
    now = datetime.now(eastern)
    end_time = now + timedelta(hours=9)
    
    start_str = now.strftime("%H:%M")
    end_str = end_time.strftime("%H:%M")

    ## scheduled data
    schedule_url = "https://api-v3.mbta.com/schedules"
    schedule_params = {
        "filter[route]": ROUTE_ID,
        "filter[min_time]": start_str,
        "filter[max_time]": end_str,
        "page[limit]": 5000,
        "include": "stop,trip"
    }
    
    sched_resp = requests.get(schedule_url, params=schedule_params)
    schedule_rows = []
    for item in sched_resp.json().get("data", []):
        rel = item["relationships"]
        attrs = item["attributes"]
        schedule_rows.append({
            "trip_id": rel["trip"]["data"]["id"] if rel.get("trip") else None,
            "stop_id": rel["stop"]["data"]["id"] if rel.get("stop") else None,
            "stop_sequence": attrs.get("stop_sequence"),
            "arrival_time": attrs.get("arrival_time"),
            "departure_time": attrs.get("departure_time"),
            "direction_id": attrs.get("direction_id"),
            "timestamp_collected": now.isoformat(),
            "schedule_type": "planned"
        })
    schedule_df = pd.DataFrame(schedule_rows)

    # real-time data pull
    pred_url = "https://api-v3.mbta.com/predictions"
    pred_params = {
        "filter[route]": ROUTE_ID,
        "page[limit]": 5000,
        "include": "stop,trip"
    }
    pred_resp = requests.get(pred_url, params=pred_params)
    pred_rows = []
    for item in pred_resp.json().get("data", []):
        rel = item["relationships"]
        attrs = item["attributes"]
        pred_rows.append({
            "trip_id": rel["trip"]["data"]["id"] if rel.get("trip") and rel["trip"]["data"] else None,
            "stop_id": rel["stop"]["data"]["id"] if rel.get("stop") and rel["stop"]["data"] else None,
            "arrival_time": attrs.get("arrival_time"),
            "departure_time": attrs.get("departure_time"),
            "status": attrs.get("status"),
            "direction_id": attrs.get("direction_id"),
            "timestamp_collected": now.isoformat(),
            "schedule_type": "realtime"
        })
    pred_df = pd.DataFrame(pred_rows)


    if not schedule_df.empty:
        if os.path.exists(OUTPUT_SCHEDULE):
            schedule_df.to_csv(OUTPUT_SCHEDULE, mode="a", header=False, index=False)
        else:
            schedule_df.to_csv(OUTPUT_SCHEDULE, index=False)
    if not pred_df.empty:
        if os.path.exists(OUTPUT_REALTIME):
            pred_df.to_csv(OUTPUT_REALTIME, mode="a", header=False, index=False)
        else:
            pred_df.to_csv(OUTPUT_REALTIME, index=False)

    print(f"[{now:%H:%M:%S}] Collected {len(schedule_df)} schedule rows and {len(pred_df)} realtime rows.")

In [4]:
## the loop
print(f"Starting MBTA Orange Line data collection for {TOTAL_DURATION_HOURS} hours")
for i in range(ITERATIONS):
    collect_mbta_data()
    if i < ITERATIONS - 1:
        print(f"Waiting {COLLECTION_INTERVAL_MINUTES} minutes before next collection...\n")
        time.sleep(COLLECTION_INTERVAL_MINUTES * 60)

print("Data Collection Complete.")

Starting MBTA Orange Line data collection for 9 hours
[14:55:04] Collected 4090 schedule rows and 539 realtime rows.
Waiting 1 minutes before next collection...

[14:56:05] Collected 4086 schedule rows and 532 realtime rows.
Waiting 1 minutes before next collection...

[14:57:06] Collected 4085 schedule rows and 543 realtime rows.
Waiting 1 minutes before next collection...

[14:58:07] Collected 4080 schedule rows and 538 realtime rows.
Waiting 1 minutes before next collection...

[14:59:09] Collected 4078 schedule rows and 544 realtime rows.
Waiting 1 minutes before next collection...

[15:00:10] Collected 0 schedule rows and 556 realtime rows.
Waiting 1 minutes before next collection...

[15:01:11] Collected 0 schedule rows and 552 realtime rows.
Waiting 1 minutes before next collection...

[15:02:11] Collected 0 schedule rows and 562 realtime rows.
Waiting 1 minutes before next collection...

[15:03:12] Collected 0 schedule rows and 554 realtime rows.
Waiting 1 minutes before next c

## Cleaning the data

In [57]:
df_schedule = pd.read_csv("orange_line_schedule_live.csv", low_memory=False)
df_realtime = pd.read_csv("orange_line_realtime_live.csv", low_memory=False)

print(df_schedule.dtypes)
print(df_realtime.dtypes)

# converting types
#     --- standardizing the IDs ----
id_columns = ['trip_id', 'stop_id']  # adjust if needed
for col in id_columns:
    df_schedule[col] = df_schedule[col].astype(str).str.strip().str.upper()
    df_realtime[col] = df_realtime[col].astype(str).str.strip().str.upper()

#     --- converting to eastern time ----
time_columns = ['departure_time', 'arrival_time']
for col in time_columns:
    df_schedule[col] = pd.to_datetime(df_schedule[col], errors='coerce', utc=True)
    df_schedule[col] = df_schedule[col].dt.tz_convert('America/New_York')
    
    df_realtime[col] = pd.to_datetime(df_realtime[col], errors='coerce', utc=True)
    df_realtime[col] = df_realtime[col].dt.tz_convert('America/New_York')

# dropping nulls
df_schedule_clean = df_schedule.dropna(subset=['trip_id', 'stop_id']).reset_index(drop=True)
df_realtime_clean = df_realtime.dropna(subset=['trip_id', 'stop_id']).reset_index(drop=True)

df_schedule_clean = df_schedule_clean.drop_duplicates(subset=['trip_id', 'stop_id', 'departure_time']).reset_index(drop=True)
df_realtime_clean = (
    df_realtime_clean.sort_values('departure_time')
    .drop_duplicates(subset=['trip_id', 'stop_id'], keep='last')
    .reset_index(drop=True)
)


print(df_schedule_clean.dtypes)
print(df_realtime_clean.dtypes)



trip_id                 int64
stop_id                 int64
stop_sequence           int64
arrival_time           object
departure_time         object
direction_id            int64
timestamp_collected    object
schedule_type          object
dtype: object
trip_id                object
stop_id                object
arrival_time           object
departure_time         object
status                 object
direction_id            int64
timestamp_collected    object
schedule_type          object
dtype: object
trip_id                                          object
stop_id                                          object
stop_sequence                                     int64
arrival_time           datetime64[ns, America/New_York]
departure_time         datetime64[ns, America/New_York]
direction_id                                      int64
timestamp_collected                              object
schedule_type                                    object
dtype: object
trip_id                       

## Merging the Data

In [58]:
df_merge_debug = pd.merge(
    df_schedule_clean,
    df_realtime_clean,
    on=['trip_id', 'stop_id'],
    how='outer',
    indicator=True
)

print("Merge counts:")
print(df_merge_debug['_merge'].value_counts())
# '_merge' values: 'left_only', 'right_only', 'both'

Merge counts:
_merge
both          4107
right_only     652
left_only        4
Name: count, dtype: int64


In [60]:
df_merged = pd.merge(
    df_schedule_clean,
    df_realtime_clean,
    on=['trip_id', 'stop_id'],
    how='inner',  # only rows present in both
    suffixes=('_schedule', '_realtime')
)

df_merged_filtered = df_merged.dropna(subset=['departure_time_schedule', 'departure_time_realtime']).reset_index(drop=True)


df_merged_filtered['delay_minutes'] = (
    df_merged_filtered['departure_time_realtime'] - df_merged_filtered['departure_time_schedule']
).dt.total_seconds() / 60

print(df_merged_filtered.head(20))
print(df_merged_filtered.shape)

df_merged_filtered.to_csv("orange_line_merged.csv", index=False)

     trip_id stop_id  stop_sequence     arrival_time_schedule  \
0   70525422   70033            170 2025-11-05 14:56:00-05:00   
1   70525422   70035            180 2025-11-05 15:00:00-05:00   
2   70525430   70029            140 2025-11-05 14:56:00-05:00   
3   70525430   70031            150 2025-11-05 14:58:00-05:00   
4   70525430   70279            160 2025-11-05 15:00:00-05:00   
5   70525430   70033            170 2025-11-05 15:02:00-05:00   
6   70525430   70035            180 2025-11-05 15:06:00-05:00   
7   70525438   70021            100 2025-11-05 14:55:00-05:00   
8   70525438   70023            110 2025-11-05 14:57:00-05:00   
9   70525438   70025            120 2025-11-05 14:58:00-05:00   
10  70525438   70027            130 2025-11-05 15:00:00-05:00   
11  70525438   70029            140 2025-11-05 15:02:00-05:00   
12  70525438   70031            150 2025-11-05 15:04:00-05:00   
13  70525438   70279            160 2025-11-05 15:06:00-05:00   
14  70525438   70033     